In [1]:
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np
from network_construction.network import source_treatment, get_Utown
import json

In [3]:
G = get_Utown()
source_nodes, treatment_nodes = source_treatment(G, 40)  # <-- Specify # starting points for treatment node algorithm

In [4]:
### MODEL PARAMETERS

# TODO: What if we just make pipe size continuous/linear

Path = {}  # Set of shortest paths from each source node i to each treatment node j
NLinks = {}  # Number of edges in each path
L = {}  # Length of each path (distance)

for i in source_nodes:
    for j in treatment_nodes:
        path = nx.shortest_path(G, source=i, target=j, weight='length')
        Path[i, j] = path
        NLinks[i, j] = len(path)-1
        L[i, j] = nx.path_weight(G, path, weight='length')
LE = {e: G.edges[e]['length'] for e in G.edges}  # Length of edge e
EL = {v: G.nodes[v]['elevation'] for v in G.nodes}  # Elevation of node v


D = [0.2, 0.25, 0.3, 0.35, 0.40, 0.45]  # Pipe diameters
CP = {0.05: 8.7, 0.06: 9.5, 0.08: 11,
                       0.1: 12.6, 0.15: 43.5, 0.2: 141,
                       0.25: 151, 0.3: 161, 0.35: 180,
                       0.4: 190, 0.45: 200}  # Cost per unit of pipe


SR = {}  # Production at Source node i
CAP = {}  # Capacity at treatment node j

for node in source_nodes:
    G.nodes[node]['production'] = .17 * 2
    SR[node] = .17 * 2

total_flow = sum(SR.values())

for node in treatment_nodes:
    G.nodes[node]['capacity'] = 100
    CAP[node] = 100

Vmin = 0.6 * 60
Vmax = 3 * 60

CE = 25  # Cost of Excavation
CB = 6  # Cost of Bedding
TR = 44000  # Fixed Cost of Treatment Plant
TRFlow = 100  # Variable Cost of Treatment
PICost = 30

PF = {'0.05': 8.7, '0.06': 9.5, '0.08': 11,
                       '0.1': 12.6, '0.15': 43.5, '0.2': 141,
                       '0.25': 151, '0.3': 161, '0.35': 180,
                       '0.4': 190, '0.45': 200}  # Fixed Cost of Piping

CT = 1000000000  # Cost of trucking
M = 1e6

Smin = 0.01
Smax = 0.1
W = 0.5  # Buffer Width

T = [0, 1]

In [ ]:
### T=0 PARAMETERS

# x_0[Path]
# y_0[j]
# z_0[G.edges]
# d_0[G.edges, D]
# el_0[G.nodes]



In [2]:
m = gp.Model()

x = m.addVars(Path.keys(), T, vtype=GRB.BINARY, name='x')  # Path ij used
y = m.addVars(treatment_nodes, T, vtype=GRB.BINARY, name='y')  # treatment at node j
z = m.addVars(G.edges, T, vtype=GRB.BINARY, name='z')  # edge e used

d = m.addVars(G.edges, D, T, vtype=GRB.BINARY, name='d')  # Pipe size s at edge e

a = m.addVars(G.edges, D, vtype=GRB.BINARY, name='a')

r = m.addVars(source_nodes, vtype=GRB.CONTINUOUS, lb=0.0, name='r')  # flow handled at trucking at edge e

Q = m.addVars(G.edges, vtype=GRB.CONTINUOUS, lb=0.0, name='Q')  # Flow in Edge e

el = m.addVars(G.nodes, T, vtype=GRB.CONTINUOUS, name='el')  # Elevation at node el_v

p = m.addVars(Path.keys(), vtype=GRB.CONTINUOUS, lb = 0.0, name='p')

m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-01-17


NameError: name 'Path' is not defined